In [ ]:
import numpy as np
import cv2

import PIL
import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning) # ignore warnings

image_shape = (224, 224)

module = hub.load("https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/classification/5")
# load the mobilenet_v2 image classification model from Tensorflow Hub

def mobilenet_v2(input):
    return module(input)

classifier = tf.keras.Sequential([
    tf.keras.layers.Input(shape = image_shape + (3,)),
    tf.keras.layers.Lambda(mobilenet_v2)
])
# classify without any changes, this will lead to errors because we need to modify the output layer as per our dataset

In [ ]:
gold_fish = Image.open("goldfish.jpg").resize(image_shape)
gold_fish

In [ ]:
gold_fish = np.array(gold_fish) / 255 
gold_fish.shape

In [ ]:
img = gold_fish[np.newaxis, ...]
img.shape

In [ ]:
y_pred = classifier.predict(img)
y_pred

In [ ]:
np.argmax(y_pred)

In [ ]:
# get all image classes present in mobilenet_v2

image_class = []

with open ('ImageNetLabels.txt') as f:
    for line in f:
        image_class.append(line.strip())

In [ ]:
image_class[np.argmax(y_pred)]

In [ ]:
import pathlib

data_dir = pathlib.Path('..//Data_Augmentation//datasets//flower_photos')
data_dir

In [ ]:
list(data_dir.glob('*/*.jpg'))[: 5]

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
image_count

In [ ]:
roses = list(data_dir.glob('roses/*'))
roses[: 5]

In [ ]:
PIL.Image.open(roses[0])

In [ ]:
# creating dictionary containing image file paths for each flower class

flower_image_path_dict = {
    'daisy' : list(data_dir.glob('daisy/*')),
    'dandelion' : list(data_dir.glob('dandelion/*')),
    'roses' : list(data_dir.glob('roses/*')),
    'sunflowers' : list(data_dir.glob('sunflowers/*')),
    'tulips' : list(data_dir.glob('tulips/*'))
}

In [ ]:
# creating label for each flower class

flower_label_dict = {
    'daisy' : 0,
    'dandelion' : 1, 
    'roses' : 2,
    'sunflowers' : 3, 
    'tulips' : 4
}

In [ ]:
# creating X and y for model 

X = []
y = []

for flower_name, flower_image_path in flower_image_path_dict.items():

    for path in flower_image_path:

        img = cv2.imread(path)
        resized_img = cv2.resize(img, image_shape)

        X.append(resized_img)
        y.append(flower_label_dict[flower_name])

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
X = X / 255

In [ ]:
X = X.astype(np.float32)  # convert from float64 to float32 to reduce memory allocation on RAM

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
img = X[2000]
plt.imshow(img)

In [ ]:
y_pred = classifier.predict(np.array([img]))

In [ ]:
np.argmax(y_pred)

In [ ]:
image_class[937]

In [ ]:
module = hub.load("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4")
# convert the module from ./classification/4 to ./feature_vector/4 to modify output layer

def mobilenet_v2(input):
    return module(input)

num_of_flowers = 5

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape = image_shape + (3, )),
    tf.keras.layers.Lambda(mobilenet_v2),

    tf.keras.layers.Dense(num_of_flowers) # modify the output layer by assigning number of nodes 
])

In [ ]:
model.compile(
  optimizer = "adam",
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
  metrics = ['accuracy']
)

model.fit(X_train, y_train, epochs = 5) 

In [ ]:
model.evaluate(X_test, y_test)